In [17]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.io import loadmat
import os
from graspy.plot import heatmap
import matplotlib.pyplot as plt
%matplotlib inline 
from tqdm import tqdm

In [79]:
data_dir = Path('/mnt/c/Users/Ronan Perry/Documents/JHU/jovo-lab/multiview/data')
embed_dir = data_dir / 'embedding'

fmri_dir = data_dir / 'raw' / 'fMRI' / 'REST_Data'
fmri_metadata_path = data_dir / 'raw' / 'fMRI' / 'SubjectsIDs_Schaefer_rest_Alone.csv'

sc_dir = data_dir / 'raw' / 'structural' / 'Deterministic_FA'
sc_metadata_path = data_dir / 'raw' / 'structural' / 'SubjectsIDs_Schaefer_Diffusion.csv'

demographics_path = data_dir / 'raw' / 'Demographics_MedicalFilter.csv'
clinical_path = data_dir / 'raw' / 'Cognitive_MedicalFilter.csv'
schaefer_coords_path = data_dir / 'supplementary' / 'Schaefer2018_400Parcels_17Networks_order.txt'

bblids_filtered_path = data_dir / 'filtered_matched_bblids.csv'

In [3]:
timestamp = 'T=336-11_31'#'T=319-11_24'
proj_mat_files = [f'fmri_gcca_projection_mat_{timestamp}.csv', f'sc_gcca_projection_mat_{timestamp}.csv', f'clinical_gcca_projection_mat_{timestamp}.csv']

In [26]:
proj_mats = [np.genfromtxt(embed_dir / file, delimiter=',') for file in proj_mat_files]

In [27]:
def ut2mat(ut, n=400):
    uidx = np.triu_indices(n)
    mat = np.zeros((n,n))
    mat[uidx] = ut
    diag = np.diagonal(mat).copy()
    mat += mat.T
    return(mat - np.diag(diag))

In [28]:
for i in [0,1]:
    proj_mats[i] = np.array([ut2mat(ut) for ut in proj_mats[i].T])
    proj_mats[i] = proj_mats[i].reshape((proj_mats[i].shape[0],-1))
    
proj_mats[2] = proj_mats[2].T

### Load Metadata

In [15]:
fmri_metadata = pd.read_csv(fmri_metadata_path)
sc_metadata = pd.read_csv(sc_metadata_path)
demographics = pd.read_csv(demographics_path)
clinical_scores = pd.read_csv(clinical_path)

In [18]:
fmri_files = [f for f in os.listdir(fmri_dir)]
fmri_ids = [f.split('_')[0] for f in os.listdir(fmri_dir)]
fmri_bblids = [fmri_metadata.query(f'scanid == {idx}')['bblid'].iloc[0] for idx in fmri_ids]

In [19]:
names = None
sc_bblids = [int(f.split('_')[0]) for f in os.listdir(sc_dir)]
sc_files = [f for f in os.listdir(sc_dir)]

#for f in os.listdir(sc_dir):
#    # one-liner to read a single variable
#    names = loadmat(sc_dir / f)['name'][0]
#    sc_files.append(loadmat(sc_dir / f)['connectivity'])

In [20]:
def get_matched_indices(x,y):
    """
    Returns indices of all shared elements of two lists
    """
    matches = set(x).intersection(set(y))
    return(([x.index(i) for i in matches], [y.index(i) for i in matches]))

In [21]:
idx_fmri, idx_sc = get_matched_indices(fmri_bblids, sc_bblids)

In [84]:
bblids_filtered = np.genfromtxt(bblids_filtered_path)

In [93]:
fmri_mats_matched = [fmri_files[i] for i in idx_fmri]
fmri_bblids_matched = [fmri_bblids[i] for i in idx_fmri]

sc_mats_matched = [sc_files[i] for i in idx_sc]
sc_bblids_matched = [sc_bblids[i] for i in idx_sc]

print(f'There are {len(np.where(not sc_bblids_matched == fmri_bblids_matched)[0])} mismatches')

bblids = sc_bblids_matched

There are 0 mismatches


In [34]:
#clinical_scores_matched = np.array([clinical_scores.query(f'bblid == {bblid}').to_numpy()[0,3:-6] for bblid in bblids_filtered])

In [87]:
fmri_dict = {int(f.split('_')[0]): f for f in os.listdir(fmri_dir)}
sc_dict = {int(f.split('_')[0]): f for f in os.listdir(sc_dir)}

In [90]:
component = 0

latents = []

for bblid in bblids_filtered:
    bblid = int(bblid)
    subj0 = [np.genfromtxt(fmri_dir / fmri_dict[bblid], delimiter=','),
             np.genfromtxt(sc_dir / sc_dict[bblid], delimiter=','),
             clinical_scores.query(f'bblid == {bblid}').to_numpy()[0,3:-6]]
    subj0_hat = [view @ proj[component,:] for view,proj in zip(subj0, proj_mats)]
    latents.append(subj0_hat)

KeyError: 80010

In [70]:
latent2 = np.array(latents)

In [73]:
latent2 = latent2.T

In [74]:
np.corrcoef(latent2)

array([[1.        , 0.21491945,        nan],
       [0.21491945, 1.        ,        nan],
       [       nan,        nan,        nan]])

In [99]:
len(bblids_filtered)

852

In [97]:
corrcoeffs = np.array([[1,0.221,0.169],[0.221,1,0.240],[0.169,0.240,1]])

In [100]:
corrcoeffs

array([[1.   , 0.221, 0.169],
       [0.221, 1.   , 0.24 ],
       [0.169, 0.24 , 1.   ]])